In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv("data.csv")

In [3]:
dataset.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
tau1     10000 non-null float64
tau2     10000 non-null float64
tau3     10000 non-null float64
tau4     10000 non-null float64
p1       10000 non-null float64
p2       10000 non-null float64
p3       10000 non-null float64
p4       10000 non-null float64
g1       10000 non-null float64
g2       10000 non-null float64
g3       10000 non-null float64
g4       10000 non-null float64
stab     10000 non-null float64
stabf    10000 non-null object
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [5]:
dataset["stabf"].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [6]:
X_data = dataset.iloc[:, :-1].values
m, n = X_data.shape

y_data = dataset.iloc[:, -1].values

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_data = encoder.fit_transform(y_data)

In [8]:
import tensorflow as tf

n_inputs  = n
n_hidden1 = 600
n_hidden2 = 330
n_outputs = 2

In [9]:
test_ratio = 0.2
test_size = int(m*test_ratio)
train_size = m - test_size

X_train = X_data[:-test_size]
X_test = X_data[-test_size:]
y_train = y_data[:-test_size]
y_test = y_data[-test_size:]

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs))
y = tf.placeholder(tf.int32, shape=(None))

In [12]:
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu)
hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu)
logits = tf.layers.dense(hidden2, n_outputs)

In [13]:
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)

In [14]:
learning_rate = 0.01

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

In [15]:
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [16]:
init = tf.global_variables_initializer()

In [17]:
n_epochs = 30
batch_size = 90
n_batches = int(np.ceil(m/batch_size))

In [18]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(train_size, size=batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    return X_batch, y_batch

In [19]:
with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        
        print(epoch, "Acc train: {:.3f}".format(acc_train), "Acc test: {:.3f}".format(acc_test))

0 Acc train: 0.922 Acc test: 0.905
1 Acc train: 0.989 Acc test: 0.928
2 Acc train: 0.944 Acc test: 0.933
3 Acc train: 0.967 Acc test: 0.943
4 Acc train: 0.933 Acc test: 0.948
5 Acc train: 0.967 Acc test: 0.961
6 Acc train: 0.967 Acc test: 0.966
7 Acc train: 0.978 Acc test: 0.970
8 Acc train: 0.967 Acc test: 0.973
9 Acc train: 1.000 Acc test: 0.975
10 Acc train: 0.989 Acc test: 0.978
11 Acc train: 1.000 Acc test: 0.979
12 Acc train: 0.989 Acc test: 0.979
13 Acc train: 0.978 Acc test: 0.983
14 Acc train: 1.000 Acc test: 0.983
15 Acc train: 1.000 Acc test: 0.983
16 Acc train: 0.989 Acc test: 0.984
17 Acc train: 0.989 Acc test: 0.984
18 Acc train: 0.989 Acc test: 0.985
19 Acc train: 1.000 Acc test: 0.986
20 Acc train: 0.989 Acc test: 0.986
21 Acc train: 1.000 Acc test: 0.985
22 Acc train: 0.989 Acc test: 0.986
23 Acc train: 0.989 Acc test: 0.988
24 Acc train: 0.989 Acc test: 0.988
25 Acc train: 1.000 Acc test: 0.985
26 Acc train: 0.989 Acc test: 0.988
27 Acc train: 1.000 Acc test: 0.989
28